In [1]:
#Importações

import pandas as pd
import binance as bn
from key import cliente

In [ ]:
#testar a conexão

cliente.get_account()

In [ ]:
import pandas as pd
import time
import pandas_ta as ta
import json
import os

# Função para carregar o estado do bot
def carregar_estado():
    estado_inicial = {
        'compras': {
            'BTCUSDT': {'compra': False, 'quantidade': 0, 'preco_compra': 0, 'comprou_extra': False, 'quantidade_extra': 0, 'preco_compra_extra': 0},
            'ETHUSDT': {'compra': False, 'quantidade': 0, 'preco_compra': 0, 'comprou_extra': False, 'quantidade_extra': 0, 'preco_compra_extra': 0},
            'BNBUSDT': {'compra': False, 'quantidade': 0, 'preco_compra': 0, 'comprou_extra': False, 'quantidade_extra': 0, 'preco_compra_extra': 0},
            'SOLUSDT': {'compra': False, 'quantidade': 0, 'preco_compra': 0, 'comprou_extra': False, 'quantidade_extra': 0, 'preco_compra_extra': 0}  # Adicionando SOL
        },
        'lucro_acumulado': 0.0
    }

    if os.path.exists('estado_bot.json'):
        with open('estado_bot.json', 'r') as f:
            estado_salvo = json.load(f)
        
        # Verificar e adicionar as chaves necessárias
        for symbol in estado_inicial['compras']:
            if symbol not in estado_salvo['compras']:
                estado_salvo['compras'][symbol] = estado_inicial['compras'][symbol]
            for key in estado_inicial['compras'][symbol]:
                if key not in estado_salvo['compras'][symbol]:
                    estado_salvo['compras'][symbol][key] = estado_inicial['compras'][symbol][key]
                    
        return estado_salvo
    
    return estado_inicial

# Função para salvar o estado do bot
def salvar_estado(estado):
    with open('estado_bot.json', 'w') as f:
        json.dump(estado, f)

# Função para atualizar dados de um símbolo
def atualizar_dados(symbol):
    try:
        df = pd.DataFrame(cliente.get_historical_klines(symbol, '1m', "60m"))  # 60 minutos de dados
        df = df.iloc[:, :6]
        df.columns = ['date_open', 'Open', 'High', 'Low', 'Close', 'Volume']
        df = df.set_index('date_open')
        df.index = pd.to_datetime(df.index, unit='ms')
        df = df.astype(float)

        df['rsi'] = ta.rsi(df['Close'], length=14)
        df['sma_20'] = ta.sma(df['Close'], length=20)
        df['sma_50'] = ta.sma(df['Close'], length=50)

        if df.shape[0] < 50:
            print(f"Erro: Dados insuficientes para {symbol}: {df.shape[0]} velas.")
            return None

        df = df.dropna()
        
        if df.empty:
            print(f"Todos os dados de {symbol} foram descartados devido a NaN.")
            return None
        
        return df
    except Exception as e:
        print(f"Erro ao atualizar dados para {symbol}: {e}")
        return None

# Função para obter o tamanho mínimo do lote e valor mínimo de notional
def get_lot_size_and_notional(symbol):
    try:
        info = cliente.get_symbol_info(symbol)
        min_qty = None
        step_size = None
        min_notional = None
        for filt in info['filters']:
            if filt['filterType'] == 'LOT_SIZE':
                min_qty = float(filt['minQty'])
                step_size = float(filt['stepSize'])
            if filt['filterType'] == 'MIN_NOTIONAL':
                min_notional = float(filt['minNotional'])
        return min_qty, step_size, min_notional
    except Exception as e:
        print(f"Erro ao buscar informações de LOT_SIZE e MIN_NOTIONAL para {symbol}: {e}")
    return None, None, None

# Função para obter o tempo do servidor
def get_server_time():
    try:
        server_time = cliente.get_server_time()
        return server_time['serverTime']
    except Exception as e:
        print(f"Erro ao obter o tempo do servidor: {e}")
        return None

# Função para executar ordem de compra ou venda
def executar_ordem(symbol, side, quantidade, server_time):
    try:
        ordem = cliente.create_order(
            symbol=symbol, 
            side=side, 
            type='MARKET', 
            quantity=f"{quantidade:.8f}", 
            recvWindow=6000, 
            timestamp=server_time
        )
        print(f"Ordem {side} realizada para {symbol}: {quantidade:.8f}")
        return True
    except Exception as e:
        print(f"Erro ao executar ordem {side} para {symbol}: {e}")
        return False
    

# Obter diferença de tempo entre servidor e sistema local
local_time = int(time.time() * 1000)
server_time = get_server_time()
time_diff = server_time - local_time if server_time else 0

# Carrega o estado do bot
estado_bot = carregar_estado()
compras = estado_bot['compras']
lucro_acumulado = estado_bot['lucro_acumulado']
# Verifica o LOT_SIZE e MIN_NOTIONAL para cada símbolo
lot_sizes = {}
for symbol in compras.keys():
    min_qty, step_size, min_notional = get_lot_size_and_notional(symbol)
    if min_notional is None:
        min_notional = 10.0
    lot_sizes[symbol] = (min_qty, step_size, min_notional)
    print(f"{symbol} - Min Qty: {min_qty:.8f}, Step Size: {step_size:.8f}, Min Notional: {min_notional:.2f}")
    
# Taxa de transação
taxa_transacao = 0.001

# Loop principal
while True:
    for symbol in compras.keys():
        df = atualizar_dados(symbol)
        if df is None:
            time.sleep(60)
            continue

        preco_atual = df['Close'].iloc[-1]
        print(f"Preço atual para {symbol}: {preco_atual:.2f} USDT")

        if not compras[symbol]['compra']:
            # Primeira compra
            if df['rsi'].iloc[-1] < 30 and preco_atual < df['sma_20'].iloc[-1]:
                quantidade_btc = 12.0 / preco_atual
                min_qty, step_size, min_notional = lot_sizes[symbol]
                quantidade_btc = max(quantidade_btc, min_qty)
                quantidade_btc = round(quantidade_btc // step_size * step_size, 8)
                if quantidade_btc >= min_qty:
                    adjusted_time = int(time.time() * 1000) + time_diff
                    if executar_ordem(symbol, 'BUY', quantidade_btc, adjusted_time):
                        compras[symbol]['compra'] = True
                        compras[symbol]['quantidade'] = quantidade_btc
                        compras[symbol]['preco_compra'] = preco_atual
                        compras[symbol]['comprou_extra'] = False
                        compras[symbol]['quantidade_extra'] = 0  # Zera quantidade extra caso tenha sido vendido

        else:
            # Compra extra
            if not compras[symbol]['comprou_extra'] and df['rsi'].iloc[-1] < 30 and preco_atual < df['sma_20'].iloc[-1]:
                quantidade_extra = 10.0 / preco_atual
                min_qty, step_size, min_notional = lot_sizes[symbol]
                quantidade_extra = max(quantidade_extra, min_qty)
                quantidade_extra = round(quantidade_extra // step_size * step_size, 8)
                if quantidade_extra >= min_qty:
                    adjusted_time = int(time.time() * 1000) + time_diff
                    if executar_ordem(symbol, 'BUY', quantidade_extra, adjusted_time):
                        compras[symbol]['quantidade_extra'] = quantidade_extra
                        compras[symbol]['preco_compra_extra'] = preco_atual
                        compras[symbol]['comprou_extra'] = True

            # Venda da compra principal
            preco_venda = (1 - taxa_transacao) * preco_atual
            preco_min_venda = compras[symbol]['preco_compra'] * (1 + taxa_transacao + 0.007)
            print(f"Preço mínimo para venda da compra principal para {symbol}: {preco_min_venda:.2f} USDT")
            if compras[symbol]['quantidade'] > 0 and preco_venda > preco_min_venda:
                adjusted_time = int(time.time() * 1000) + time_diff
                if executar_ordem(symbol, 'SELL', compras[symbol]['quantidade'], adjusted_time):
                    lucro_transacao = (preco_venda - compras[symbol]['preco_compra']) * compras[symbol]['quantidade']
                    lucro_acumulado += lucro_transacao
                    compras[symbol]['compra'] = False
                    print(f"Lucro da venda principal para {symbol}: {lucro_transacao:.2f} USDT")

            # Venda da compra extra
            if compras[symbol]['quantidade_extra'] > 0:
                preco_min_venda_extra = compras[symbol]['preco_compra_extra'] * (1 + taxa_transacao + 0.007)
                print(f"Preço mínimo para venda da compra extra para {symbol}: {preco_min_venda_extra:.2f} USDT")
                if preco_venda > preco_min_venda_extra:
                    adjusted_time = int(time.time() * 1000) + time_diff
                    if executar_ordem(symbol, 'SELL', compras[symbol]['quantidade_extra'], adjusted_time):
                        lucro_transacao = (preco_venda - compras[symbol]['preco_compra_extra']) * compras[symbol]['quantidade_extra']
                        lucro_acumulado += lucro_transacao
                        compras[symbol]['quantidade_extra'] = 0
                        compras[symbol]['comprou_extra'] = False
                        print(f"Lucro da venda extra para {symbol}: {lucro_transacao:.2f} USDT")

        estado_bot['compras'] = compras
        estado_bot['lucro_acumulado'] = lucro_acumulado
        salvar_estado(estado_bot)

    time.sleep(160)


BTCUSDT - Min Qty: 0.00001000, Step Size: 0.00001000, Min Notional: 10.00
ETHUSDT - Min Qty: 0.00010000, Step Size: 0.00010000, Min Notional: 10.00
BNBUSDT - Min Qty: 0.00100000, Step Size: 0.00100000, Min Notional: 10.00
SOLUSDT - Min Qty: 0.00100000, Step Size: 0.00100000, Min Notional: 10.00
Preço atual para BTCUSDT: 75992.00 USDT
Preço mínimo para venda da compra principal para BTCUSDT: 76234.16 USDT
Preço mínimo para venda da compra extra para BTCUSDT: 76087.40 USDT
Preço atual para ETHUSDT: 2915.25 USDT
Preço atual para BNBUSDT: 591.38 USDT
Preço mínimo para venda da compra principal para BNBUSDT: 606.61 USDT
Preço mínimo para venda da compra extra para BNBUSDT: 605.90 USDT
Preço atual para SOLUSDT: 197.60 USDT
Preço mínimo para venda da compra principal para SOLUSDT: 201.89 USDT
